In [1]:
%matplotlib inline

datapath = "../../data/dl1/obj_neural_code/all_flat/"
repath   = "../../data/dl1/obj_neural_code/rep/nofinetune/"
N = 1440

In [2]:
from time import time
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import scipy.spatial.distance
from keras.models import Model
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
from keras.models import Model

idx = [0,5,10,17,24,31,33,35,37]
# lambda_1
# maxpooling2d_1
# maxpooling2d_2
# maxpooling2d_3
# maxpooling2d_4
# maxpooling2d_5
# dense_1
# dense_2
# dense_3

def get_models(idx,base):
    layers  = []
    for i in range(len(idx)):
        print(base.layers[idx[i]].name)
        layers.append( base.layers[idx[i]] )

    models = []
    outshapes = []
    for layer in layers:
        models.append( Model(input=base.input,
                             output=base.get_layer(layer.name).output) )
        outshapes.append(list( layer.output_shape ))
        
        
    print('Done.')
    return layers,models,outshapes

In [4]:
batch_size = 60
batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=False)
nbatches = int(N/batch_size)
print('N.of batches ' + str(nbatches))

Found 1440 images belonging to 1 classes.
N.of batches 24


In [5]:
base = vgg.model
layers,models,outshapes = get_models(idx,base)

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_3
Done.


In [6]:
for i in range(len(outshapes)):
    outshapes[i][0] = N
    print(outshapes[i])

[1440, 3, 224, 224]
[1440, 64, 112, 112]
[1440, 128, 56, 56]
[1440, 256, 28, 28]
[1440, 512, 14, 14]
[1440, 512, 7, 7]
[1440, 4096]
[1440, 4096]
[1440, 1000]


In [7]:
D = []
D_sq = []
for layer,model in zip(layers,models):
    print('Processing model : ' + layer.name)
    R = []
    batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=False)
    ti = time()
    for n in range(nbatches):
        imgs,_ = batches.next()       
        R.append(model.predict(imgs))
    R = np.asarray(R, dtype=float) 
    outshape = list(layer.output_shape)
    outshape[0] = N
    R.shape = tuple(outshape)
    #print(R.shape)
    np.save(repath +  layer.name, R)
    
    # compute distances
    R.shape = R.shape[0], -1
    d = scipy.spatial.distance.pdist(R, 'euclidean')
    d_sq = scipy.spatial.distance.squareform(d, force='no', checks=True)
    D.append(d)
    D_sq.append(d_sq)
    te = time() - ti
    print(te) 
    
np.save(repath + 'D', D)
np.save(repath + 'D_sq', D_sq)
print('Done.')

Processing model : lambda_1
Found 1440 images belonging to 1 classes.
228.577388048
Processing model : maxpooling2d_1
Found 1440 images belonging to 1 classes.
1179.22591114
Processing model : maxpooling2d_2
Found 1440 images belonging to 1 classes.
610.578848124
Processing model : maxpooling2d_3
Found 1440 images belonging to 1 classes.
326.505110979
Processing model : maxpooling2d_4
Found 1440 images belonging to 1 classes.
190.782716036
Processing model : maxpooling2d_5
Found 1440 images belonging to 1 classes.
86.6446008682
Processing model : dense_1
Found 1440 images belonging to 1 classes.
58.667714119
Processing model : dense_2
Found 1440 images belonging to 1 classes.
59.2492880821
Processing model : dense_3
Found 1440 images belonging to 1 classes.
55.3338198662
Done.
